In [153]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/fashion',one_hot=True,validation_size=0)
train_X, train_Y, testX, testY = mnist.train.images,mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting data/fashion\train-images-idx3-ubyte.gz
Extracting data/fashion\train-labels-idx1-ubyte.gz
Extracting data/fashion\t10k-images-idx3-ubyte.gz
Extracting data/fashion\t10k-labels-idx1-ubyte.gz


In [154]:
sess =tf.InteractiveSession()

In [155]:
def b_variable(shape):
    '''设置一个偏置值'''
    initial = tf.constant(0.1 ,shape=shape)
    return tf.Variable(initial)

In [156]:
def convolution(input_X, shape, filters, padding='same'): #前后shape不会改变 只改变通道数即filters
    input_X = tf.reshape(input_X, shape)
    return tf.layers.conv2d(input_X, filters, (2, 2),padding=padding, activation=tf.nn.relu6, kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))

In [157]:
def pooling(input_X, shape, filters=(2, 2), strides=(2, 2), padding='same'): #这里选择的max_pooling 我也不知道为什么average_pooling不常用
    input_X = tf.reshape(input_X, shape)
    return tf.layers.max_pooling2d(input_X, filters, strides, padding=padding)

In [158]:
input_X = tf.placeholder("float", [None, 784])
y_ = tf.placeholder("float", [None, 10])
#第一层卷积加池化[-1,28,28,1]->[-1,28,28,32]->[-1,14,14,32]
#with tf.variable_scope('conv1'):
b1 = b_variable([32])
conv1 = convolution(input_X, [-1, 28, 28, 1], filters=32)+b1
#with tf.name_scope("pool1"):
conv1 = pooling(conv1,[-1,28,28,32]) 

#第二层卷积加池化[-1,14,14,32]->[-1,14,14,64]->[-1,7,7,64]
#with tf.variable_scope("conv2"):
b2 = b_variable([64])
conv2 = convolution(conv1, [-1, 14, 14, 32], filters=64)+b2
#with tf.name_scope("pool2"):
conv2 = pooling(conv2, [-1,14,14,64])

#第三层卷积[-1,7,7,64]->[-1,7,7,128] 
#with tf.variable_scope("conv3"):
b3 = b_variable([128])
conv3 = convolution(conv2, [-1,7,7,64], filters=128)+b3



In [159]:
# 重塑出
shape = np.prod(conv3.get_shape().as_list()[1:])
fc1 = tf.reshape(conv3,[-1,shape])

In [160]:
#全连接层
#with tf.variable_scope("flatten"):
fc1 = tf.contrib.layers.fully_connected(fc1,1024,activation_fn=tf.nn.relu6)
fc1 = tf.nn.dropout(fc1, keep_prob=0.5)
fc2 = tf.contrib.layers.fully_connected(fc1, 512, activation_fn=tf.nn.relu6)
logits_ = tf.contrib.layers.fully_connected(fc2, 10, activation_fn=None)     #logistic层
logits_ = tf.identity(logits_, name='logits_')
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_, labels=y_))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
#optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.FtrlOptimizer(learning_rate=learning_rate)
correct_pred = tf.equal(tf.argmax(logits_,1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [161]:
#train_writer = tf.summary.FileWriter('graphs/')
#train_writer.add_graph(tf.get_default_graph())
sess.run(tf.global_variables_initializer())

In [131]:
for i in range(15000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0 and i !=0:
        train_accuracy = accuracy.eval(feed_dict={input_X:batch[0] ,y_:batch[1]})
        print("samples %d,train's accuracy: %f"%(i,train_accuracy))
    optimizer.run(feed_dict={input_X:batch[0] ,y_:batch[1]})

samples 100,train's accuracy: 0.140000
samples 200,train's accuracy: 0.140000
samples 300,train's accuracy: 0.760000
samples 400,train's accuracy: 0.660000
samples 500,train's accuracy: 0.680000
samples 600,train's accuracy: 0.660000
samples 700,train's accuracy: 0.820000
samples 800,train's accuracy: 0.820000
samples 900,train's accuracy: 0.860000
samples 1000,train's accuracy: 0.820000
samples 1100,train's accuracy: 0.820000
samples 1200,train's accuracy: 0.840000
samples 1300,train's accuracy: 0.760000
samples 1400,train's accuracy: 0.800000
samples 1500,train's accuracy: 0.820000
samples 1600,train's accuracy: 0.860000
samples 1700,train's accuracy: 0.820000
samples 1800,train's accuracy: 0.960000
samples 1900,train's accuracy: 0.940000
samples 2000,train's accuracy: 0.780000
samples 2100,train's accuracy: 0.840000
samples 2200,train's accuracy: 0.840000
samples 2300,train's accuracy: 0.840000
samples 2400,train's accuracy: 0.880000
samples 2500,train's accuracy: 0.900000
samples 2

KeyboardInterrupt: 

In [121]:
saver=tf.train.Saver()
saver.save(sess,r'C:\Users\JIN\Desktop\machine\homework1\session15000')

'C:\\Users\\JIN\\Desktop\\machine\\homework1\\sess1.csv'

In [119]:
# overfit后的模型  test集固定在0.9072上
batch = mnist.train.next_batch(50)
train_accuracy = accuracy.eval(feed_dict={input_X:batch[0] ,y_:batch[1]})
print("samples train_test,train's accuracy: %f"%(train_accuracy))

testSet = mnist.test.next_batch(10000)
c = accuracy.eval(feed_dict={input_X: testSet[0], y_: testSet[1]})
print("test accuracy %g" % c)

samples train_test,train's accuracy: 1.000000
test accuracy 0.9072


In [136]:
saver=tf.train.Saver()
saver.save(sess,r'C:\Users\JIN\Desktop\machine\homework1\session10000')

'C:\\Users\\JIN\\Desktop\\machine\\homework1\\session10000'

In [135]:
batch = mnist.train.next_batch(50)
train_accuracy = accuracy.eval(feed_dict={input_X:batch[0] ,y_:batch[1]})
print("samples train_test,train's accuracy: %f"%(train_accuracy))

testSet = mnist.test.next_batch(10000)
c = accuracy.eval(feed_dict={input_X: testSet[0], y_: testSet[1]})
print("test accuracy %g" % c)

samples train_test,train's accuracy: 0.880000
test accuracy 0.9012
